In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [14]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewDeaths"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [15]:
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,Date
0,USA,245080,5.0,6075.0,5.0,10403.0,228602,5421.0,740.0,18.0,2020-04-02 23:01:14.506057
1,Italy,115242,0.0,13915.0,0.0,18278.0,83049,4053.0,1906.0,230.0,2020-04-02 23:01:14.506057
2,Spain,112065,0.0,10348.0,0.0,26743.0,74974,6092.0,2397.0,221.0,2020-04-02 23:01:14.506057
3,Germany,84794,0.0,1107.0,0.0,22440.0,61247,3936.0,1012.0,13.0,2020-04-02 23:01:14.506057
4,France,59105,0.0,5387.0,0.0,12428.0,41290,6399.0,905.0,83.0,2020-04-02 23:01:14.506057
...,...,...,...,...,...,...,...,...,...,...,...
201,St. Vincent Grenadines,2,0.0,0.0,0.0,1.0,1,0.0,18.0,0.0,2020-04-02 23:01:14.506057
202,Sierra Leone,2,0.0,0.0,0.0,0.0,2,0.0,0.3,0.0,2020-04-02 23:01:14.506057
203,Papua New Guinea,1,0.0,0.0,0.0,0.0,1,0.0,0.1,0.0,2020-04-02 23:01:14.506057
204,Timor-Leste,1,0.0,0.0,0.0,0.0,1,0.0,0.8,0.0,2020-04-02 23:01:14.506057


In [16]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Deaths/1M pop,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
1,Italy,115242,0.0,13915.0,0.0,18278.0,83049,4053.0,1906.0,230.0,2020-04-02 23:01:14.506057,12.070013,42.796626,32193.0,0.432237,0.120746,0.121364,0.014654,Last 8 columns are added by Roaya
2,Spain,112065,0.0,10348.0,0.0,26743.0,74974,6092.0,2397.0,221.0,2020-04-02 23:01:14.506057,-3.647550,40.244487,37091.0,0.278990,0.092339,0.139829,0.012912,Last 8 columns are added by Roaya
176,China,81620,4.0,3322.0,4.0,76571.0,1727,379.0,57.0,2.0,2020-04-02 23:01:14.506057,103.819074,36.561765,79893.0,0.041581,0.040701,0.301189,0.012259,Last 8 columns are added by Roaya
4,France,59105,0.0,5387.0,0.0,12428.0,41290,6399.0,905.0,83.0,2020-04-02 23:01:14.506057,2.618787,47.824905,17815.0,0.302386,0.091143,0.067161,0.006121,Last 8 columns are added by Roaya
5,Iran,50468,0.0,3160.0,0.0,16711.0,30597,3956.0,601.0,38.0,2020-04-02 23:01:14.506057,54.274070,32.575033,19871.0,0.159026,0.062614,0.074912,0.004691,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,Benin,13,0.0,0.0,0.0,1.0,12,0.0,1.0,0.0,2020-04-02 23:01:14.506057,2.327853,9.641760,1.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
115,Aruba,60,0.0,0.0,0.0,1.0,59,0.0,562.0,0.0,2020-04-02 23:01:14.506057,-69.982677,12.520880,1.0,0.000000,0.000000,0.000004,0.000000,Last 8 columns are added by Roaya
147,Dominica,12,0.0,0.0,0.0,0.0,12,0.0,167.0,0.0,2020-04-02 23:01:14.506057,-61.357726,15.439470,0.0,0.000000,0.000000,0.000000,0.000000,Last 8 columns are added by Roaya
149,Greenland,10,0.0,0.0,0.0,2.0,8,0.0,176.0,0.0,2020-04-02 23:01:14.506057,-41.341911,74.710513,2.0,0.000000,0.000000,0.000008,0.000000,Last 8 columns are added by Roaya


In [17]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [18]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [19]:
corona_json

'[{"Country":"USA","TotalCases":245080,"NewCases":5.0,"TotalDeaths":6075.0,"NewDeaths":5.0,"TotalRecovered":10403.0,"ActiveCases":228602,"CriticalCases":5421.0,"Tot\\u00a0Cases\\/1M pop":740.0,"Deaths\\/1M pop":18.0,"Date":1585868474506,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":16478.0,"ClosedDeathRate":0.3686733827,"CurrentDeathRate":0.0247878244,"Weight":0.0621204182,"DeathRateXWeight":0.00153983,"notes":"Last 8 columns are added by Roaya"},{"Country":"Italy","TotalCases":115242,"NewCases":0.0,"TotalDeaths":13915.0,"NewDeaths":0.0,"TotalRecovered":18278.0,"ActiveCases":83049,"CriticalCases":4053.0,"Tot\\u00a0Cases\\/1M pop":1906.0,"Deaths\\/1M pop":230.0,"Date":1585868474506,"Longitude":12.07001339,"Latitude":42.79662641,"ClosedCases":32193.0,"ClosedDeathRate":0.4322368217,"CurrentDeathRate":0.1207459086,"Weight":0.1213644023,"DeathRateXWeight":0.014654255,"notes":"Last 8 columns are added by Roaya"},{"Country":"Spain","TotalCases":112065,"NewCases":0.0,"TotalDeaths":